# Connectivity matrix extraction example

In [ ]:
import obi_one as obi
from pathlib import Path

In [ ]:
input_root = Path("../../data/tiny_circuits")
output_root = Path("../../../../obi-output/connectivity_matrix_extraction")

### Run connectivity matrix extraction

In [ ]:
connectivity_matrix_extractions_scan_config = obi.ConnectivityMatrixExtractionScanConfig(
                    initialize=obi.ConnectivityMatrixExtractionScanConfig.Initialize(
                        circuit=[obi.Circuit(name="N_10__top_nodes_dim6", path=str(input_root / "N_10__top_nodes_dim6" / "circuit_config.json")),
                                 obi.Circuit(name="N_10__top_rc_nodes_dim2_rc", path=str(input_root / "N_10__top_rc_nodes_dim2_rc" / "circuit_config.json"))],
                        edge_population="S1nonbarrel_neurons__S1nonbarrel_neurons__chemical",
                        # node_attributes=None,
                        node_attributes=("synapse_class", "layer", "mtype", "etype", "x", "y", "z"),
                    )
)

grid_scan = obi.GridScan(form=connectivity_matrix_extractions_scan_config, output_root=str(output_root), coordinate_directory_option="VALUE")

In [ ]:
grid_scan.execute()
obi.run_task_for_single_configs_of_generated_scan(grid_scan)

In [ ]:
# Display the intermediary data
grid_scan.multiple_value_parameters(display=True)
grid_scan.coordinate_parameters(display=True)
# grid_scan.coordinate_instances(display=True)

### Check & plot extracted connectivity matrices

In [ ]:
# Check matrices
from conntility.connectivity import ConnectivityMatrix
import matplotlib.pyplot as plt

for circuit in connectivity_matrix_extractions_scan_config.initialize.circuit:
    cmat = ConnectivityMatrix.from_h5(output_root / circuit.name / "connectivity_matrix.h5")
    print(f"Connectivity matrix extracted from '{circuit.name}': {cmat.vertices.shape[0]} neurons, {cmat.matrix.size} connections, {cmat.matrix.sum()} synapses")

    plt.figure(figsize=(4, 3))
    plt.imshow(cmat.dense_matrix, interpolation="nearest")
    plt.xlabel("Post-synaptic neurons")
    plt.ylabel("Pre-synaptic neurons")
    plt.title(circuit.name, fontweight="bold")
    plt.colorbar(label="#Synapses")
    plt.tight_layout()
    plt.show()